<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 7: Matched Filtering Part II - Frequency Domain</h1>


<a name='section_7_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P7.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_1">P7.1 Introduction to Fitting in the Frequency Domain</a>
        </td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">no problems</td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_2">P7.2 Analysis of Noisy Car Horn Data</a>
        </td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_7_2">P7.2 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_3">P7.3 Calculating a Better Chi-square for LIGO Model</a>
        </td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_7_3">P7.3 Problems</a></td>
    </tr>
</table>

<h3>Learning Objectives</h3>

In this Pset we will continue our analysis using matched filtering, this time in the frequency domain.

<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

In [ ]:
#>>>RUN: P7.0-runcell00

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/P06' >> .git/info/sparse-checkout
!git pull origin main

<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook.

In [ ]:
#>>>RUN: P7.0-runcell01

!pip install lmfit
!pip install playsound
!pip install soundfile

In [ ]:
#>>>RUN: P7.0-runcell02

import numpy as np                 #https://numpy.org/doc/stable/
import matplotlib.pyplot as plt    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
from scipy.io.wavfile import write   #https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html

from lmfit import Model, Parameters #https://lmfit.github.io/lmfit-py/parameters.html
                                    #https://lmfit.github.io/lmfit-py/model.html#lmfit.model.Model
import scipy.stats as stats         #https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy.stats import chisquare   #https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: P7.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_7_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P7.1 Introduction to Fitting in the Frequency Domain</h2>   

| [Top](#section_7_0) | [Previous Section](#section_7_0) | [Problems](#problems_7_1) | [Next Section](#section_7_2) |


<h3>Overview</h3>

**Note:** There are no problems in this section. Review and run the cells to understand later sections.

Previously, we studied the procedure of fitting in the time domain, including discovering some of its limitations. Now, we're going to do a similar process in the frequency domain. However, instead of performing a fit, we're going compare the data to a template of a merger signal which we will generate.

Let's start by making the template in the time domain using the same function as before. The code below creates two sets of y values, one for the same parameters as before (`y_true`), and one with parameters that are similar to the true values but slightly shifted (`y_temp`). Note that the `y_temp` array uses a `TIME_TRUE` value of 0. In the plot, the time for the `y_true` array is shifted so that the two overlap.

In what follows, we will call the `y_true` array the "data" and the `y_temp` array the "template".

In [ ]:
#>>>RUN: P7.1-runcell01

np.random.seed(0x98a09fe)

def complicated_model_fn(x, time, lambda_plus, lambda_minus, max_amp, omega_0, omega_max, omega_sigma):
    omega = (omega_max - omega_0) * (np.exp(-np.minimum(x - time, 0)**2 / omega_sigma)) + omega_0
    lambdas = np.array([lambda_plus if xvalue > time else lambda_minus for xvalue in x])
    amplitude = max_amp * np.exp(-abs(x - time) / lambdas)
    return amplitude * np.cos(omega * (x-time))

LAMBDA_PLUS_TRUE = 1.0
LAMBDA_MINUS_TRUE = 4
MAX_AMP_TRUE = 1.2
OMEGA_0_TRUE = 3.0
OMEGA_MAX_TRUE = 6.0
OMEGA_SIGMA_TRUE = 4.0
TIME_TRUE = 50.0

sample_spacing = 0.1
xi       = np.arange(-128, 128, sample_spacing)#times
xi_shift = np.arange( -26, 230, sample_spacing)#times

yi_temp = complicated_model_fn(xi_shift, 0, 1.5, 3.5, 1.0, 3.2, 5.5, 3.5)
yi_true = complicated_model_fn(xi, TIME_TRUE+TIME_SHIFT, LAMBDA_PLUS_TRUE, LAMBDA_MINUS_TRUE, MAX_AMP_TRUE,
                               OMEGA_0_TRUE, OMEGA_MAX_TRUE, OMEGA_SIGMA_TRUE)

#Alternative approach
#template_mask = np.where((xi > -15) & (xi < 5))
#data_mask = np.where((xi > TIME_TRUE-15) & (xi < TIME_TRUE+5))#
#plt.plot(xi[data_mask]-TIME_TRUE, yi_true[data_mask], label="True")

template_mask = np.where((xi > -128) & (xi < 128))
data_mask     = np.where((xi > -128) & (xi < 128))
plt.plot(xi[data_mask], yi_true[data_mask], label="True")
plt.plot(xi[template_mask], yi_temp[template_mask], label="Template")
plt.xlabel("Time (s)")
plt.ylabel("Strain")
plt.legend();

In order to work in the frequency domain, we need to start by taking a Fourier transform of our template and data. Curiously, even though the two samples have an identical functional form, as well as very similar input parameters, the FFT outputs are very different, especially for the imaginary parts. As you can see in the 3rd plot, the real parts of the two FFTs also have some differences.

In [ ]:
#>>>RUN: P7.1-runcell02

fs = int(1/(xi[1] - xi[0]))

data_fft = np.fft.fft(yi_true)
template_fft = np.fft.fft(yi_temp)

freq = np.fft.fftfreq(xi.shape[0])*fs

plt.figure(figsize=(16, 5))
plt.title("FFT of data")
plt.plot(freq, data_fft.imag, label='imaginary')
plt.plot(freq, data_fft.real, label='real')
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.ylim(-20, 20)
plt.legend()
plt.show()

plt.figure(figsize=(16, 5))
plt.title("FFT of template")
plt.plot(freq, template_fft.real, label='real')
plt.plot(freq, template_fft.imag, label='imaginary')
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.ylim(-20, 20)
plt.legend()
plt.show()

plt.figure(figsize=(16, 5))
plt.title("Real part of FFT of data and template")
plt.plot(freq, data_fft.real, label='data')
plt.plot(freq, template_fft.real, label='template')
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.ylim(-20, 20)
plt.legend()
plt.show()


Now, suppose we want to do the same thing we did in the time domain case: shift the template by a time $t$. What happens to the FFT? Consider the integral definition of a Fourier transform (as a substitute for the discrete case):

$$\mathcal{F}(\omega) = \int dt f(t)  e^{-2\pi i\omega t}$$

So the FFT for a function shifted by $\Delta t$ is

$$\int dt f(t - \Delta t)  e^{-2\pi i\omega t} = \int dt' f(t')  e^{-2\pi i\omega (t'+\Delta t)} = e^{-2\pi i \omega \Delta t}\mathcal{F}(\omega).$$

The time-shifted FFT is just a constant multiplicative factor (albeit an imaginary number) times the initial template FFT!


So, suppose our data is indeed just our template, shifted by some time $\Delta t$. We can find $\Delta t$ from the data FFT $\mathcal{D}$ and the template FFT $\mathcal{T}$ using

$$\mathcal{D}(\omega)=e^{-2\pi i\omega \Delta t} \mathcal{T}(\omega) \implies \frac{\mathcal{D}(\omega)}{\mathcal{T}(\omega)}=e^{-2\pi i\omega \Delta t} = \frac{\mathcal{D}(\omega) \mathcal{T}^*(\omega)}{|\mathcal{T}|^2(\omega)}.$$

The inverse Fourier transform (IFFT) of the constant exponential factor is a delta function at $\Delta t$:

$$f(t) = \int d\omega e^{-2\pi i\omega \Delta t} e^{2\pi i\omega t} = \delta(\Delta t - t).$$

Consequently, the peak of the IFFT of 
$$f(\omega) =\mathcal{D}(\omega) \mathcal{T}^*(\omega)$$
must also be centered at $\Delta t$.

There are some extra complications involved here, but discussion above gives the gist of what happens. For more in-depth information, see the <a href="https://www.gw-openscience.org/s/events/GW150914/LOSC_Event_tutorial_GW150914.html" target="_blank">LIGO tutorial from which this code is inspired</a>.

Let's compute $f(\omega)$ as given in the formula above. We call $f(\omega)$ `optimal_fft`. Then, we take the IFFT and call it `optimal_time`.

In [ ]:
#>>>RUN: P7.1-runcell03

fftout=np.fft.fft(yi_temp)
optimal_fft = data_fft * template_fft.conjugate() / np.abs(fftout**2)
plt.plot(freq,np.abs(optimal_fft))
plt.title("optimal_fft")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

#note the 2 is here b/c of the fft
optimal_time = 2*np.fft.ifft(optimal_fft)*fs
plt.plot(xi,optimal_time.real)
plt.title("optimal_time")
plt.xlabel("Time")
plt.ylabel("Power")
plt.show()

We should really plot some sort of signal to noise ratio, not `optimal_time` itself. Let's fix the ratio so that if the data is just noise, we have a ratio of one. We record the ratio as `SNR` and plot it.


In [ ]:
#>>>RUN: P7.1-runcell04

df = np.abs(freq[1] - freq[0])
#compute the resolution sigma^2=(|S|^2/PSD)*Delta t
sigmasq = 2*(template_fft * template_fft.conjugate() / fftout**2).sum() * df
sigma = np.sqrt(np.abs(sigmasq))
SNR = abs(optimal_time) / (sigma)

plt.figure()
plt.plot(xi, SNR)
plt.xlabel('Offset time (s)')
plt.ylabel('SNR');

<a name='section_7_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P7.2 Analysis of Noisy Car Horn Data</h2>   

| [Top](#section_7_0) | [Previous Section](#section_7_1) | [Problems](#problems_7_2) | [Next Section](#section_7_3) |


<h3>Setup for Problems</h3>

In what follows, we will use a frequency based matched filtering algorithm to solve a different kind of problem. You will load two audio files: one containing a specific kind of car horn and another one containing street noise with this car horn sound inserted.

Note that there are several car horn sounds embedded in the street noise sample but, if executed correctly, the frequency based matched filtering should be able to find the specific car horn sound we are looking for!

In [ ]:
#>>>RUN: P7.2-runcell01

import soundfile as sf
import time
from IPython.display import Audio, display

# Load the data into arrays
yi_true, fs = sf.read('data/P06/street_noise.wav') 
yi_temp, fs = sf.read('data/P06/car_horn.wav')
yi_true = yi_true[:,0]
yi_temp = yi_temp[:,0]

print(min,max)
#play these guys
min=int(22*(len(yi_true)/60))
max=int(26*(len(yi_true)/60))

def play(iArray,iFS):
    sf.write('data/P06/tmp.flac', iArray, iFS)
    display(Audio('data/P06/tmp.flac',autoplay=False))

print("street_noise")
play(yi_true[min:max],fs)

print("car horn")
play(yi_temp,fs)

# Compresses the data to a smaller size by averaging 5 element chunks together
# Note that this does not change the data, just decreases its resolution
yi_true     = np.array(yi_true)
yi_true_avg = np.average(yi_true.reshape(-1, 5), axis=1)



# Make the template the same length as the signal by filling it with 0's
# This is needed for the fft analysis to work
yi_temp = np.concatenate((yi_temp, np.zeros(5*int(len(yi_temp)+1)-len(yi_temp))))
yi_temp_avg = np.average(yi_temp.reshape(-1, 5), axis=1)
yi_temp_avg = np.concatenate((yi_temp_avg, np.zeros(len(yi_true_avg)-len(yi_temp_avg))))


# Create time array, from 0 to 60 second (the length of the street noise recording)
# Time array also has the same resolution as the street noise, in order for fft analysis to work
xi = np.linspace(0, 60, len(yi_true_avg))

#Let's plot
plt.plot(xi,yi_true_avg,label='merged sound')
plt.plot(xi,yi_temp_avg,label='car horn')
plt.xlabel('time(s)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

<a name='problems_7_2'></a>   

| [Top](#section_7_0) | [Restart Section](#section_7_2) | [Next Section](#section_7_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 7.2.1</span>

The code above loaded two datasets: one is a template containing the car horn and the other is true data containing car horn embedded into street noise. Using the same approach as in the previous section, first plot the real and imaginary parts of the FFT for both datasets. Also, plot the real parts of the FFT of the template compared to the noisy data. Fill in the missing code to accomplish this.

Consider the plots of the template and the data in the frequency domain. Which sound sample has a greater range of frequencies (a spectrum with power more broadly and continuously spread over frequency)?

A. Template

B. Data

C. They look very similar

In [ ]:
#>>>PROBLEM: P7.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.


#copy and paste the FFT
fs = #YOUR CODE HERE
data_fft = #YOUR CODE HERE
template_fft = #YOUR CODE HERE
freq = #YOUR CODE HERE


plt.figure(figsize=(16, 5))
plt.title("FFT of data")
plt.plot(freq, data_fft.real, label='real')
plt.plot(freq, data_fft.imag, label='imaginary')
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
#plt.ylim(-20, 20)
plt.legend()
plt.show()

plt.figure(figsize=(16, 5))
plt.title("FFT of template")
plt.plot(freq, template_fft.real, label='real')
plt.plot(freq, template_fft.imag, label='imaginary')
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
#plt.ylim(-20, 20)
plt.legend()
plt.show()

plt.figure(figsize=(16, 5))
plt.plot(freq,data_fft.real,label='data')
plt.plot(freq,template_fft.real,label='signal')
plt.xlabel('freq')
plt.ylabel('N')
plt.legend()
plt.show()


>#### Follow-up 7.2.1a (ungraded)
>   
>Why does this answer make sense? Think about all the noise in the data that doesn't exist in the template.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 7.2.2</span>

When looking at the FFT of the LIGO template shown earlier, there was a large difference between the real and imaginary parts of the frequency space. Is there a large difference in the real and imaginary parts of the car-horn template?


A. Yes

B. No


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 7.2.3</span>

At what frequency (in Hz) does the power spectral density (PSD) spectrum of the data distribution peak? Enter your answer as a number with precision 1 (i.e. the closest integer).

Hint: Take the absolute value of the FFT$^{2}$ of the data to get the PSD spectrum. Use the helpful command `np.argmax()` to find the index of the maximum element of the PSD. Use this index to find the frequency.

In [ ]:
#>>>PROBLEM: P7.2.3

freq_psd = #YOUR CODE HERE
freq_psd_max = #YOUR CODE HERE

plt.plot(freq,freq_psd,label='psd')
plt.title("PSD of data")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()
print("Frequency at which the PSD is at its maximum: ", freq_psd_max)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 7.2.4</span>

At what time (in seconds) does the car horn signal modeled in the template occur in the data? Enter your answer as a number with precision 1e-1.

Hint: Follow the example from what was done in Section P7.1 to find the time shift between the LIGO signal template and the data. Again, use `np.argmax()` to find the index of the maximum element of the `SNR`, and find the corresponding element of the time data array.

In [ ]:
#>>>PROBLEM: P7.2.4


optimal_fft = #YOUR CODE HERE
optimal_time = #YOUR CODE HERE

df = #YOUR CODE HERE
sigmasq = #YOUR CODE HERE
sigma =#YOUR CODE HERE
SNR = #YOUR CODE HERE

time_SNR_max = #YOUR CODE HERE

plt.figure()
plt.plot(xi, SNR)
plt.xlabel('Offset time (s)')
plt.ylabel('SNR')
plt.show()

print("Time at which SNR peaks: ", time_SNR_max)

>#### Follow-up 7.2.4a (ungraded)
>   
>The robustness of this approach is pretty impressive if you have a template that you are looking for in a sea of data. Listen to the audio files more closely. What are the other, smaller, peaks in the spectrum corresponding to? When do you hear other car horn noises in the data? Can you hear the template noise in the data at the correct time?

>#### Follow-up 7.2.4b (ungraded)
>   
>Change the data, perhaps by adding more noise or a different kind of noise, or by shifting the signal to a different time. What changes in the matched filter result?


>#### Follow-up 7.2.4c (ungraded)
>   
>The runtime of the frequency domain matched filtering process was much faster than the time domain process. But we should not compare the two approaches directly because we tested against a template for the frequency version and performed a whole fit for the time version.
>
>If, instead of fitting, we used a fixed template for the time version as well, computing a $\chi^2$ for every offset $t$ between the signal and the template and plotted $\chi^2$ as a function of $t$, which method do you think would be faster now? Frequency or time?



<a name='section_7_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P7.3 Calculating a Better Chi-square</h2>   

| [Top](#section_7_0) | [Previous Section](#section_7_2) | [Problems](#problems_7_3) |


<h3>Overview</h3>

Now, we are going to combine Fourier analysis and fitting of the time series information, to get a better notion of the $\chi^{2}$. 

For this, we'll use similar data to what was used for fitting in the time domain, namely a merger signal embedded in 10 sine waves. 

In [ ]:
#>>>RUN: P7.3-runcell01

np.random.seed(0x98a09fe)

def complicated_model_fn(x, time, lambda_plus, lambda_minus, max_amp, omega_0, omega_max, omega_sigma):
    omega = (omega_max - omega_0) * (np.exp(-np.minimum(x - time, 0)**2 / omega_sigma)) + omega_0
    lambdas = np.array([lambda_plus if xvalue > time else lambda_minus for xvalue in x])
    amplitude = max_amp * np.exp(-abs(x - time) / lambdas)
    return amplitude * np.cos(omega * (x-time))


params_min_max = {
    'lambda_plus': (0.1, 5),
    'lambda_minus': (0.1, 5),
    'max_amp': (0, 2),
    'omega_0': (0, 5),
    'omega_max': (0, 10),
    'omega_sigma': (0, 5),
}

def get_param_random_value(p_min,p_max):
    #get a uniformly distributed random value between p_min and p_max
    #return a float
    return p_min + (p_max - p_min) * np.random.random(1)[0]

def model_and_random_parameters(t):
    model = Model(complicated_model_fn)
    params = Parameters()
    params.add('time', value=t, vary=False)
    for p, (p_min, p_max) in params_min_max.items():
        value = get_param_random_value(p_min,p_max)
        params.add(p, min=p_min, max=p_max, value=value)
    return model, params


LAMBDA_PLUS_TRUE = 1.0
LAMBDA_MINUS_TRUE = 4
MAX_AMP_TRUE = 1.2
OMEGA_0_TRUE = 3.0
OMEGA_MAX_TRUE = 6.0
OMEGA_SIGMA_TRUE = 4.0
TIME_TRUE = 50.0

xi = np.linspace(TIME_TRUE-15, TIME_TRUE+5, 200)
true_yi = complicated_model_fn(xi, TIME_TRUE, LAMBDA_PLUS_TRUE, LAMBDA_MINUS_TRUE, MAX_AMP_TRUE,
                               OMEGA_0_TRUE, OMEGA_MAX_TRUE, OMEGA_SIGMA_TRUE)

NUMBER_SINES_TO_ADD = 10

noise_frequencies = 0.5 + 7 * np.random.random(NUMBER_SINES_TO_ADD)
noise_phases = 2 * np.pi * np.random.random(NUMBER_SINES_TO_ADD)
noise_amplitudes = 2 * MAX_AMP_TRUE / NUMBER_SINES_TO_ADD * np.random.random(NUMBER_SINES_TO_ADD)
    # The above line sets noise amplitudes so that the sum of all the noise amplitudes is on average
    # equal to the maximum amplitude of the signal.

plt.plot(xi, true_yi)
plt.title("True Signal")
plt.xlabel("Time (s)")
plt.ylabel("Strain")
plt.show()

sample_spacing = 0.1
xi = np.arange(-128, 128, sample_spacing)#times
yi = np.zeros_like(xi)#data

#Adding Noise
for freq, phase, amplitude in zip(noise_frequencies, noise_phases, noise_amplitudes):
    yi += amplitude * np.sin(phase + freq * xi)

#Adding Data
signal= complicated_model_fn(xi, TIME_TRUE, LAMBDA_PLUS_TRUE, LAMBDA_MINUS_TRUE, MAX_AMP_TRUE,
                               OMEGA_0_TRUE, OMEGA_MAX_TRUE, OMEGA_SIGMA_TRUE)
yi+=signal

plt.plot(xi, yi)
plt.plot(xi, signal)
plt.title("Signal plus noise")
plt.xlabel("Time (s)")
plt.ylabel("Strain")
plt.show()

plt.plot(xi, yi)
plt.plot(xi, signal)
plt.title("Signal plus noise")
plt.xlim(35,55)
plt.xlabel("Time (s)")
plt.ylabel("Strain")
plt.show()

<h3>Plotting the Fit from Before (time-domain)</h3>

Here we plot the fit, defining a slightly different function that will also return the data arrays. Run it multiple times to find the minimum $\chi^2$ value and corresponding maximum $\chi^2$ probability.

**Note:** Since we will use this fit going forward, be sure to run it until the best fit (minimum $\chi^2$) is found and **make sure** that a fit resulting in this lowest value is the **last fit** that you run. To ensure that you end on the correct fit, it's best **not** to use a `for` loop in this case.

In [ ]:
#>>>RUN: P7.3-runcell02

#Getting the fit (defining a slightly different function than previously)

import lmfit


def get_signal_indices(xi, t, t_before, t_after):
    #use np.where() to return a 1D the relevant indices
    #note, the result of np.where() will be a tuple
    return np.where((xi > t - t_before) & (xi < t + t_after))

def fit_once_apply(t, t_before, t_after, weight=1.0):
    data_indices = get_signal_indices(xi, t, t_before, t_after)
    data_x = xi[data_indices]
    data_y = yi[data_indices]
    weights = np.ones(len(data_x))*weight
    model, params = model_and_random_parameters(t)
    result = model.fit(data_y, params, x=data_x,weights=weights)
    fitted_y = model.eval(x=data_x,params=result.params)
    result.plot()
    print("Fit chi2 value: ", result.chisqr)
    print("Fit chi2 probability: ",1-stats.chi2.cdf(result.chisqr,result.nfree))
    return data_x,data_y,fitted_y

t_before = 5
t_after = 2
unc=0.2
fit_x,d_y, fit_y = fit_once_apply(TIME_TRUE, t_before, t_after, 1./unc)
plt.show()

Now, your job is to compute a more meaningful $\chi^{2}$ value by taking the above data and fitted prediction and transforming them into Fourier space and plotting them together.

<a name='problems_7_3'></a>   

| [Top](#section_7_0) | [Restart Section](#section_7_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 7.3.1</span>

As a first step, take the data and the fitted function in the time range they were fitted, and plot them in Fourier space. At approximately what frequency value (in Hz) does the amplitude decay to negligible levels? Enter your answer as a number with precision $\pm 0.01$ Hz.


In [ ]:
#>>>PROBLEM: P7.3.1

fftdata = #YOUR CODE HERE
fftfit  = #YOUR CODE HERE

freq = np.fft.fftfreq(fit_x.shape[0])*sample_spacing

freq    = freq   [0:freq.shape[0]//2]
fftfit  = fftfit [0:fftfit.shape[0]//2]
fftdata = fftdata[0:fftdata.shape[0]//2]

#print(fftdata.shape,fftfit.shape)
plt.plot(freq,fftdata.real,label='data')
plt.plot(freq,fftfit.real,label='fit')
plt.xlabel('freq')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 7.3.2 </span>

Now, we want to compute the noise as a function of frequency. We can do this by taking a region where there is no signal and computing the standard deviation of our samples in Fourier space. Notice that the code does this in each of the same frequency bins that were used to compute the FFT, and uses `np.stddev(axis=0)` to compute the standard deviation of a 2D array.

Run the code below. What does this noise spectrum look like? Choose from the following options:

- A uniform distribution across all frequencies.
- A decaying exponential function, peaked at low frequencies.
- An increasing exponential function, peaked at high frequencies.
- A function with multiple peaks in the low frequency range, decreasing in the high frequency range.
- A function with multiple peaks in the high frequency range, decreasing in the low frequency range.


In [ ]:
#>>>PROBLEM: P7.3.2

def fft_region(t, t_before, t_after):
    data_indices = get_signal_indices(xi, t, t_before, t_after)
    data_x = xi[data_indices]
    data_y = yi[data_indices]
    fft_sample = np.fft.fft(data_y)
    fft_sample=fft_sample[0:fft_sample.shape[0]//2]
    return fft_sample

tsample=np.arange(-128, 128,10)

t_before = 5
t_after = 2

ffts=np.array([])
for t in tsample:
    if abs(t-TIME_TRUE) > 10:
        pfft=fft_region(t, t_before, t_after)
        if pfft.shape[0] == 35:
            ffts = np.append(ffts,pfft)
            
ffts   = np.reshape(ffts,(len(ffts)//35,35))
stddev = ffts.std(axis=0)
plt.plot(freq,stddev)
plt.xlabel('freq')
plt.ylabel('std deviation')
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 7.3.3 </span>

Finally, make a plot of the Fourier transforms of the data and the best time-domain fit, similar to what was done at the beginning of this section, but now in frequency space. Include error bars on each data point equal to the standard deviation of the corresponding frequency found in P7.3.1 above.

What is the $\chi^{2}$ value between the data and the fit (i.e. $\sum (\textrm{data}-\textrm{fit})^2/\textrm{unc}^2$) and its associated probability? Note that the number of degrees of freedom is the number of bins minus the number of fit parameters, the latter being 7 in this case.

Enter your answer as a list of numbers `[chi2, chi2_prob]` with precision 1e-1.  

**NOTE: make sure you perform this analysis with respect to the time-domain fit that yielded the lowest $\chi^2$ value.**

>#### Follow-up 7.3.3a (ungraded)
>   
>Why is the fit probability found using this frequency-domain technique more meaningful than the p-value obtained from the time-domain fit itself?